In [17]:
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import price_data as price
import ccxt
import time
import chart


In [5]:
ftx = ccxt.ftx({
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True,
})

In [12]:
daily=price.get_price_data('1d',symbol='ETH/USD')
hourly=price.get_price_data('1h',symbol='ETH/USD')

In [7]:
results_df = pd.read_csv('data/results.csv')
results_df.iloc[results_df['success_rate'].idxmax()]

Unnamed: 0      368.0000
ema              28.0000
channel          18.0000
success_rate      0.9375
Name: 368, dtype: float64

In [8]:
float(next(filter(lambda x: x['coin']=='USD',ftx.fetch_balance()['info']['result']))['total'])

64.35567923

In [9]:
next(filter(lambda x: x['future']=='ETH-PERP',ftx.fetch_positions()))

{'future': 'ETH-PERP',
 'size': '0.002',
 'side': 'sell',
 'netSize': '-0.002',
 'longOrderSize': '0.0',
 'shortOrderSize': '0.0',
 'cost': '-4.283',
 'entryPrice': '2141.5',
 'unrealizedPnl': '0.0',
 'realizedPnl': '7.9855152',
 'initialMarginRequirement': '1.0',
 'maintenanceMarginRequirement': '0.03',
 'openSize': '0.002',
 'collateralUsed': '4.283',
 'estimatedLiquidationPrice': '44443.140197357745'}

In [10]:
trade_amount=10/hourly.iloc[-1]['close']
trade_amount

0.004670496473775163

In [11]:
int(time.time())/3600 == int(time.time())//3600

False

In [21]:
daily=price.get_price_data('1d',symbol='ETH/USD')
hourly=price.get_price_data('1h',symbol='ETH/USD')
gradient=chart.get_gradient(chart.get_ema(daily,28))

In [24]:
print(daily['unix'].max())

1625961600000


In [27]:
print(max(gradient.index))

1625961600000
